In [4]:
'''
Collection of functions to audit and clean street type and post codes
'''

import xml.etree.cElementTree as ET
from collections import defaultdict
import re

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE) 
# regulat expression to find street names

postal_re = re.compile('\d{5}(-\d{4})?$')
# regular expression to find full post code

short_postal_re = re.compile(r'(\d{5})')  
# regular expression to find 5 digit post code

expected_street_types = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Terrace"]


def audit_street_type(street_types, street_name):
    """Build defaultdict of unexpected street types.
    Args:
        street_types (dict): defaultdict(set) of unexpected street types.
        street_name (str): street name data.
    """
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected_street_type:
            street_types[street_type].add(street_name)
            
def audit_post_codes(post_codes, post_code):
    """Build set of unexpected post codes.
    Args:
        post_codes (set): unexpected post codes.
        post_code (str): post code data.
    """
    m = post_code_re.match(post_code)
    if m:
        post_codes.add(post_code)
            
            
def is_street_name(elem):
    """Determine whether street name data.
    Args:
        elem (obj): element found using ET.iterparse().
    Returns:
        True for success, False otherwise.
    """
    return (elem.attrib['k'] == "addr:street")

def is_post_code(elem):
    """Determine whether post code data.
    Args:
        elem (obj): element found using ET.iterparse().
    Returns:
        True for success, False otherwise.
    """
    return (elem.attrib['k'] == "addr:postcode")

def audit(osmfile):
    """Audit street type and post code data.
    Args:
        osmfile (obj): OSM (XML) file to audit.
    Returns:
        street_types (dict): defaultdict(set) of unexpected street types.
        post_codes (set): unexpected post codes.
    """
    osm_file = open(osmfile, "rb")
    street_types = defaultdict(set)
    post_codes = set()
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
                elif is_post_code(tag):
                    audit_post_codes(post_codes, tag.attrib['v'])
    osm_file.close()
    return street_types, post_codes
